<a href="https://colab.research.google.com/github/manneryzach/mcgill-physics-hack2021/blob/main/chnaldi_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chnaldi Patterns
Chnaldi figures are obtained when a certain piece of material, usually circular of square, is vibrating in a particular mode. the nodes and anti-nodes 
formed by the oscillation make complex but symmetrical patterns all over the surface that can be seen if a fine grain material is sprinkled on top of the surface.

Our project consists of finding those same patterns for boundary conditions more elaborate than the usal square. In fact, the user will make is own shape and the code will give him back the first resonance patterns. 


In [ ]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [13]:
from dolfin import *